# Python Design Patterns

## Behavioral Design Patterns

### Strategy Pattern
#### Pattern Summary
* general description
  * different strategies need to be applied to a class method/function
* use case:
  + Item class needs to apply a variety of different discount strategy
  + discount strategies are implemented as different functions
  + a strategy function is passed to the class constructor to instantiate a class object applying that strategy
* advantages:
  + open/close principle: easy to introduce new strategies without changing client code
  + isolation: isolate the specific implementation details of the algorithms from the client's code
  + encapsulation: data structure used for implementing the algorithm are completely encapsulated in Strategy class. Therefore, the implementation of an algorithm can be changed without affecting the Context class 
    + here Context class is the interface/abstract class of the strategy function
  + Run-time switch: application can switch the strategies at the run-time
* disadvantages:
  + creating extra objects: needs to create and maintain both strategy class instance and strategy function object
  + awareness among clients: need to understand the difference between strategies to apply them
  + increases the complexity: no need to implement if there are only a few algorithms to implement

#### Implement Strategy Pattern
* in this implementation, the self object is passed to strategy functions

In [14]:
# code from https://www.geeksforgeeks.org/strategy-method-python-design-patterns/
"""Strategy class"""
class Strategy:

    """Constructor function with price and discount"""

    def __init__(self, price, discount_strategy = None):
        
        """take price and discount strategy"""
        
        self.price = price
        self.discount_strategy = discount_strategy
        
    
    """A separate function for price after discount"""

    def price_after_discount(self):
        
        if self.discount_strategy:
            discount = self.discount_strategy(self)
        else:
            discount = 0
            
        return self.price - discount

    def __repr__(self):

        statement = "Price: {}, price after discount: {}"
        return statement.format(self.price, self.price_after_discount())

""" Strategy functions dedicated to On Sale Discount"""
def on_sale_discount(order):
    return order.price * 0.25 + 20

"""function dedicated to 20 % discount"""
def twenty_percent_discount(order):
    return order.price * 0.20

In [15]:
"""Cleint Code"""

"""using default strategy"""
print(Item(20000))

"""with discount strategy as 20 % discount"""
print(Item(20000, discount_strategy = twenty_percent_discount))

"""with discount strategy as On Sale Discount"""
print(Item(20000, discount_strategy = on_sale_discount))

Price: 20000, price after discount: 20000
Price: 20000, price after discount: 16000.0
Price: 20000, price after discount: 14980.0


* Implementation using dynamic type creation
  * this implementation uses the dynamic method bound to Strategy class
  * the self object is directly defined in strategy functions
  * types.MethodType() takes two arguments, the function object, and the object the function is bound to

In [16]:
import types #Import the types module

class Strategy:
    """The Strategy Pattern class"""
    
    def __init__(self, price, function=None):
        self.price = price
        
        #If a reference to a function is provided, replace the execute() method with the given function
        if function:
            self.execute = types.MethodType(function, self)
            
    def execute(self): #This gets replaced by another version if another strategy is provided.
        """The defaut method that prints the name of the strategy being used"""        
        return self.price
    
    def __repr__(self):
        statement = "Price: {}, price after discount: {}"
        return statement.format(self.price, self.execute())

#Replacement method 1
def on_sale_discount(self):
    return self.price * 0.25 + 20    

#Replacement method 2    
def twenty_percent_discount(self):
    return self.price * 0.20

In [17]:
"""Cleint Code"""

"""using default strategy"""
print(Item(20000))

"""with discount strategy as 20 % discount"""
print(Item(20000, discount_strategy = twenty_percent_discount))

"""with discount strategy as On Sale Discount"""
print(Item(20000, discount_strategy = on_sale_discount))

Price: 20000, price after discount: 20000
Price: 20000, price after discount: 16000.0
Price: 20000, price after discount: 14980.0


* Template for Strategic Pattern

In [18]:
# from https://github.com/LinkedInLearning/python-design-patterns-2422610/blob/04_08e/strategy_final.py
import types #Import the types module

class Strategy:
    """The Strategy Pattern class"""
    
    def __init__(self, function=None):
        self.name = "Default Strategy"
        
        #If a reference to a function is provided, replace the execute() method with the given function
        if function:
            self.execute = types.MethodType(function, self)
            
    def execute(self): #This gets replaced by another version if another strategy is provided.
        """The defaut method that prints the name of the strategy being used"""
        print("{} is used!".format(self.name))

#Replacement method 1
def strategy_one(self):
    print("{} is used to execute method 1".format(self.name))

#Replacement method 2    
def strategy_two(self):
    print("{} is used to execute method 2".format(self.name))
    
#Let's create our default strategy
s0 = Strategy()
#Let's execute our default strategy
s0.execute()

#Let's create the first varition of our default strategy by providing a new behavior
s1 = Strategy(strategy_one)
#Let's set its name
s1.name = "Strategy One"
#Let's execute the strategy
s1.execute()

s2 = Strategy(strategy_two)
s2.name = "Strategy Two"
s2.execute()


Default Strategy is used!
Strategy One is used to execute method 1
Strategy Two is used to execute method 2


### Observer Pattern
#### Pattern summary
* general description
  * allows you to define or create a subscription mechanisms to send notifications to multiple objects about any new event happens to the object they are observing.
  * the subject maintains the sinble copy of the state information observed by multiple observers, and notify them once the state information is changed or updated.
  * this pattern defines one to many dependencies between subject and observer objects
  * this is also called sub/pub because subject publish updateds of state information to observers
  + singleton is relaated to this pattern
* use case
  + there is a core temperature of reactors at a powerplant
  + there are many registered observers that will be notified whenever temperature changes
* advantages:
  + open/closed principle: introducing new observers is easy
  + establishes relationship: easy to establish the relationships at the runtime between objects
  + loose coupling between subjects and observers. There is no need to modify subject to add or remove observers

#### Implement Observer Pattern
* define an abstact class of Subject with the following methods:
  + attach
  + detach
  + notify
* concrete subject classes inherit Subject class (Core class)
  + maintain temp property
 * Observer concrete classes implementing update() method that can be called by notify() method from concrete Subject classes 

In [21]:
class Subject(object): #Represents what is being 'observed'

    def __init__(self):
        self._observers = [] # This where references to all the observers are being kept
        # Note that this is a one-to-many relationship: there will be one subject to be observed by multiple _observers

    def attach(self, observer):
        if observer not in self._observers: #If the observer is not already in the observers list
            self._observers.append(observer) # append the observer to the list

    def detach(self, observer): #Simply remove the observer
        try:
            self._observers.remove(observer)
        except ValueError:
            pass

    def notify(self, modifier=None):
        for observer in self._observers: # For all the observers in the list
            if modifier != observer: # Don't notify the observer who is actually updating the temperature 
                observer.update(self) # Alert the observers!

class Core(Subject): #Inherits from the Subject class

    def __init__(self, name=""):
        Subject.__init__(self)
        self._name = name #Set the name of the core
        self._temp = 0 #Initialize the temperature of the core

    @property #Getter that gets the core temperature
    def temp(self):
        return self._temp

    @temp.setter #Setter that sets the core temperature
    def temp(self, temp):
        self._temp = temp
        self.notify() #Notify the observers whenever somebody changes the core temperature

class TempViewer:
    def update(self, subject): #Alert method that is invoked when the notify() method in a concrete subject is invoked
        print("Temperature Viewer: {} has Temperature {}".format(subject._name, subject._temp))

#Let's create our subjects
c1 = Core("Core 1")
c2 = Core("Core 2")

#Let's create our observers
v1 = TempViewer()
v2 = TempViewer()

#Let's attach our observers to the first core
c1.attach(v1)
c1.attach(v2)

#Let's change the temperature of our first core
c1.temp = 80
c1.temp = 90


Temperature Viewer: Core 1 has Temperature 80
Temperature Viewer: Core 1 has Temperature 80
Temperature Viewer: Core 1 has Temperature 90
Temperature Viewer: Core 1 has Temperature 90


### Iterator Pattern
#### Pattern Summary
* general description
  + Iterator Pattern is used to access elements of an aggregate object sequentially without exposing its underlying implementation
* use case
  + create a custom iterator based on built-in Python iterator: zip()
  + the iterator is used to traverse German counting words only upto a certain point
* advantages:
  + single responsibility: extract the huge algorithms in to separate classes in the iterator method so that aggregate classes don't have to also manage the iteration functions
  + open/closed principal:passing the new iterators and collections into the client code will not break the code
  + easy to use interface: it makes the interface simple to use and also supports the variations in the traversal of the collections
* disadvantages:
  + increases complexity: may introduce unnecessary complexity for simple applications
  + decreases efficiency: accessing elements directly is a much better option as compared to accessing elements using the iterator in terms of efficiency

#### Implement Iterator Pattern
* we define a generator function that will generate the German counting words, and use the for loop to take advantage of the iterator automatically provided by the for loop to traverse the generated numbers

In [25]:
def count_to(count):
    """Our iterator implementation"""
    
    #Our list
    numbers_in_german = ["eins", "zwei", "drei", "vier", "funf"]

    #Our built-in iterator
    #Creates a tuple such as (1, "eins")
    iterator = zip(range(count), numbers_in_german)
    
    #Iterate through our iterable list
    #Extract the German numbers
    #Put them in a generator called number
    for position, number in iterator:
        
        #Returns a 'generator' containing numbers in German
        yield number 

#Let's test the generator returned by our iterator
for num in count_to(3):
    print("{}".format(num))

for num in count_to(4):
    print("{}".format(num))
    

eins
zwei
drei
eins
zwei
drei
vier


## Structural Patterns

### Adapter pattern
#### Pattern Summary
* general description
  + adpater an interface into another one that a client is expecting
* use case
  + we have two objects: Korean and British that implements differnt speak functions: speak_Korean() and speak_English()
  + client only want to call a uniform method: speak()
* advantages:
  + principle of single responsibility: we separate teh concrete code from the primary logic of client. Client only need to call the uniform method
  + flexibility and reusability of the code
  + less complicated class: client class is not complicated by having to use a different interface and can use ploymorphism to swap between different implementatio of adapters
  + open/close principle: we can introduce new adapter classes into the code without violating the open/closed principle
* disadvantages:
  + complexity of code: new classes, objects and interfaces are introduced
  + adaptability: most of the time, we require many adaptations with the adaptee chain to reach the compatibility we want

#### Implement Adapter Pattern
* the following cell contains the implementation using some python features
* we passed the adaptee object to the adapter class dunder init()
* we also passed the methods that need to be translated using \*\*adapted_method, where keys are the translated method names and values are the objects of the corresponding methods in the original adaptee class
* we then use \_\_dict\_\_.update(adapted_method) to update the instance object attribute dictionary
* we then define the dunder getattr() to intercept the attribute getter to retrieve the other attributes from the passed object
* notice that in line 43-44, we transfer the methods themselves to keyword argument speak without calling the methods

In [26]:
class Korean:
    """Korean speaker"""
    def __init__(self):
        self.name = "Korean"

    def speak_korean(self):
        return "An-neyong?"

class British:
    """English speaker"""
    def __init__(self):
        self.name = "British"

    #Note the different method name here!
    def speak_english(self):
        return "Hello!"	

class Adapter:
    """This changes the generic method name to individualized method names"""

    def __init__(self, object, **adapted_method):
        """Change the name of the method"""
        self._object = object

        #Add a new dictionary item that establishes the mapping between the generic method name: speak() and the concrete method
        #For example, speak() will be translated to speak_korean() if the mapping says so
        self.__dict__.update(adapted_method)

    def __getattr__(self, attr):
        """Simply return the rest of attributes!"""
        return getattr(self._object, attr)
        
#List to store speaker objects
objects = []

#Create a Korean object
korean = Korean()

#Create a British object
british =British()

#Append the objects to the objects list
objects.append(Adapter(korean, speak=korean.speak_korean))
objects.append(Adapter(british, speak=british.speak_english))


for obj in objects:
    print("{} says '{}'\n".format(obj.name, obj.speak()))


Korean says 'An-neyong?'

British says 'Hello!'



### Decorator Pattern
#### Pattern Summary
* general description
  + allows you to dynamically attach new behaviors/features to existing object without changing their structure or using subclassing
  + generall, we implement by placing these objects inside the wrapper objects that contains the behviors
  + It is much easier to implement Decorator pattern in Python because of its built-in feature
  + note that Decorator pattern is different from inheritance because the new feature is added only to that particular object, not to the entire subclass
* use case
  + we need add HTML decoration tags around the input text
  + we want to combine these decorators in all possible orders 
* advantages
  + single responsibility principle: it is easy to divide a monolithic class which implements many possible variants of behavior into several classes using Decorator Pattern
  + Runtime Responsibilities: we can easily add or remove the responsibilities from an object at runtime
  + subclassing: The decorator pattern is an alternative to subclassing. Subclassing adds behavior at compile time, and the changes affect all instances of the original class. Decorating can provide new behavior at runtime for individual objects
* disadvantages
  + complicated decorators: it can be complicated to have decorators keep track of other decorators
  + ugly configuration: large number of code of layer might make the configurations ugly


#### Implement Decorator Pattern
* the Docorator Pattern was implemented using function decorator in the following cell

In [27]:
# version 1 using function decorator without parameters
from functools import wraps

def make_blink(function):
    """Defines the decorator"""

    #This makes the decorator transparent in terms of its name and docstring
    @wraps(function)

    #Define the inner function
    def decorator():
        #Grab the return value of the function being decorated
        ret = function() 

        #Add new functionality to the function being decorated
        return "<blink>" + ret + "</blink>"

    return decorator

#Apply the decorator here!
@make_blink
def hello_world():
    """Original function! """

    return "Hello, World!"

#Check the result of decorating
print(hello_world())

#Check if the function name is still the same name of the function being decorated
print(hello_world.__name__)

#Check if the docstring is still the same as that of the function being decorated
print(hello_world.__doc__)


<blink>Hello, World!</blink>
hello_world
Original function! 


In [28]:
# version 2 using function decorator with input parameter
from functools import wraps

def make_tag(tag):    
    def make_blink(function):
        """Defines the decorator"""

        #This makes the decorator transparent in terms of its name and docstring
        @wraps(function)

        #Define the inner function
        def decorator():
            #Grab the return value of the function being decorated
            ret = function() 

            #Add new functionality to the function being decorated
            return f"<{tag}>" + ret + f"</{tag}>"

        return decorator
    return make_blink

#Apply the decorator here!
@make_tag("blink")
def hello_world():
    """Original function! """

    return "Hello, World!"

#Check the result of decorating
print(hello_world())

#Check if the function name is still the same name of the function being decorated
print(hello_world.__name__)

#Check if the docstring is still the same as that of the function being decorated
print(hello_world.__doc__)


<blink>Hello, World!</blink>
hello_world
Original function! 


## Creational Pattern

### Factory Pattern

#### Pattern Summary
* general description
  + allows an interface or a class to create an object and encapsulate the creation logic 
  + objects are created without exposing the logic to client 
  + client use the same common interface to create new type of object
  + used when there are uncertainties in types of objects to use and needs to decide what classes to create at runtime
* use case
  + we have different classes of pets, including dog and cat and may add other new classes later
  + To create instances of different classes of pets, we define the factory method: get_pet() function
  + this function encapsulates the details of how the different pet instances are created, and clients only need to tell which type of pets (dog or cat) to the function, and then get a pet
* advantages
  + allows to easily add new types of products to the factory method without disturbing the client code
  + avoides tight coupling between the product classes and creator classes and objects

#### Implement Factory Pattern
* we take the advantage of Python's feature by using python dictionary to create and fetch the appropriate instances
* the factory method used the class type argument sent to the factory method to create objects of the input classes
* instead of using multiple if/else branches, we use python dictionary to store and retrieve instance objects

In [2]:
class Dog:

    """A simple dog class"""

    def __init__(self, name):
        self._name = name

    def speak(self):
        return "Woof!"

class Cat:

    """A simple cat class"""

    def __init__(self, name):
        self._name = name

    def speak(self):
        return "Meow!"

def get_pet(pet="dog"):

    """The factory method"""

    pets = dict(dog=Dog("Hope"), cat=Cat("Peace"))

    return pets[pet]

d = get_pet("dog")

print(d.speak())

c = get_pet("cat")

print(c.speak())

Woof!
Meow!


### Abstract Factory Pattern
#### Pattern Summary
* general description
  + it is built on factory pattern
  + allows you to produce a family of related objects without specifying their concrete clases, which is decided at runtime
  + provides the encapsulation of a group of individual factories
  + this pattern tryies to abstract the creation of objects depending on the logic, business, platform etc.
* use case
  + we have a pet factory, which consists of a dog factory and a cat factory
  + each of the dog and cat factory creates dog and cat objects, as well as the related products, such as dog food and cat food ect.
  + It is clear that we have a family of pet factories. We have the following components
    + abstract factory: pet factory
    + concrete factory: dog factory and cat factory
    + concrete products: dog and dog food, cat and cat food
* advantages 
  + particularly useful when client doesn't know exactly what type to create
  + easy to introduce new variants of the products without breaking the existing client code
  + products which we get from the factory are surely compatible with each other in terms of the shared code in the abstract factory class
* disadvantages:
  + complicatd code with many small files

#### Implement Abstract Factory Pattern
* different from other languages such as java where an abstract factory class is used to define all the "shared" methods, and an abstract factory method implemented in the concrete subclasses, Python doesn't need to use such an inheritance method
* in the following code example, the concrete factory instance is directly sent to the abstract factory's constructor

In [3]:
class Dog:
    """One of the objects to be returned"""

    def speak(self):
        return "Woof!"

    def __str__(self):
        return "Dog"


class DogFactory:
    """Concrete Factory"""

    def get_pet(self):
        """Returns a Dog object"""
        return Dog()

    def get_food(self):
        """Returns a Dog Food object"""
        return "Dog Food!"


class PetStore:
    """ PetStore houses our Abstract Factory """

    def __init__(self, pet_factory=None):
        """ pet_factory is our Abstract Factory """

        self._pet_factory = pet_factory


    def show_pet(self):
        """ Utility method to display the details of the objects retured by the DogFactory """

        pet = self._pet_factory.get_pet()
        pet_food = self._pet_factory.get_food()

        print("Our pet is '{}'!".format(pet))
        print("Our pet says hello by '{}'".format(pet.speak()))
        print("Its food is '{}'!".format(pet_food))


#Create a Concrete Factory
factory = DogFactory()

#Create a pet store housing our Abstract Factory
shop = PetStore(factory)

#Invoke the utility method to show the details of our pet
shop.show_pet()


Our pet is 'Dog'!
Our pet says hello by 'Woof!'
Its food is 'Dog Food!'!


### Singleton Pattern
#### Pattern Summary
* general description
  + when you allow only one object to be created from a class
  + it is like a global variable in an object-oriented way
  + Borg or monostate singleton design pattern, multiple instances are allowed, but they all share a single copy of a state  attribute
  + notice that all Python modules are singleton, and is shared by mutiple objects
* use case
  + acts as an information cached shared by multiple objects
* advantages
  + initializations: An object created by the singleton pattern is initalized only when it is requested for the first time
  + access to the object: global access to the instance of the object
  + count of instances: in singleton, method classes can't have more than one instnace
* disadvantages:
  + multithread environment: it is not easy to use in a multithread environment, because we have to take care that the multithread wouldn't create a singleton object several times
  + single responsibility principle: it is solving two problems at a single time
  + unit test process: introducing global state to the applications makes unit test difficult

#### Implement Singleton Pattern
* The first implementation is Borg pattern
* the second one uses dobule checked Locking singleton pattern
* the third one uses a static method to check if the instance has been created

In [4]:
class Borg:
    """Borg pattern making the class attributes global"""
    _shared_data = {} # Attribute dictionary

    def __init__(self):
        self.__dict__ = self._shared_data # Make it an attribute dictionary

        
class Singleton(Borg): #Inherits from the Borg class
    """This class now shares all its attributes among its various instances"""
    #This essenstially makes the singleton objects an object-oriented global variable

    def __init__(self, **kwargs):
        Borg.__init__(self)
        self._shared_data.update(kwargs) # Update the attribute dictionary by inserting a new key-value pair 

    def __str__(self):
        return str(self._shared_data) # Returns the attribute dictionary for printing

#Let's create a singleton object and add our first acronym
x = Singleton(HTTP="Hyper Text Transfer Protocol")
# Print the object
print(x) 

#Let's create another singleton object and if it refers to the same attribute dictionary by adding another acronym.
y = Singleton(SNMP="Simple Network Management Protocol")
# Print the object
print(y)


{'HTTP': 'Hyper Text Transfer Protocol'}
{'HTTP': 'Hyper Text Transfer Protocol', 'SNMP': 'Simple Network Management Protocol'}


In [5]:
# Double Checked Locking singleton pattern
import threading


class SingletonDoubleChecked(object):

    # resources shared by each and every
    # instance

    __singleton_lock = threading.Lock()
    __singleton_instance = None

    # define the classmethod
    @classmethod
    def instance(cls):

        # check for the singleton instance
        if not cls.__singleton_instance:
            with cls.__singleton_lock:
                if not cls.__singleton_instance:
                    cls.__singleton_instance = cls()

        # return the singleton instance
        return cls.__singleton_instance


# main method
if __name__ == '__main__':

    # create class X
    class X(SingletonDoubleChecked):
        pass

    # create class Y
    class Y(SingletonDoubleChecked):
        pass

    A1, A2 = X.instance(), X.instance()
    B1, B2 = Y.instance(), Y.instance()
    assert A1 is not B1
    assert A1 is A2
    assert B1 is B2

    print('A1 : ', A1)
    print('A2 : ', A2)
    print('B1 : ', B1)
    print('B2 : ', B2)


A1 :  <__main__.X object at 0x000001B611D8A5C0>
A2 :  <__main__.X object at 0x000001B611D8A5C0>
B1 :  <__main__.Y object at 0x000001B611D88730>
B2 :  <__main__.Y object at 0x000001B611D88730>


In [7]:
# classic implementation of Singleton Design pattern
class Singleton:

    __shared_instance = 'GeeksforGeeks'

    @staticmethod
    def getInstance():
        """Static Access Method"""
        if Singleton.__shared_instance == 'GeeksforGeeks':
        	Singleton()
        return Singleton.__shared_instance

    def __init__(self):
        """virtual private constructor"""
        if Singleton.__shared_instance != 'GeeksforGeeks':
            raise Exception("This class is a singleton class !")
        else:
            Singleton.__shared_instance = self


# main method
if __name__ == "__main__":

    # create object of Singleton Class
    obj = Singleton()
    print(obj)

    # pick the instance of the class
    obj = Singleton.getInstance()
    print(obj)
